In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [2]:
import pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

/home/igor/miniconda3/envs/env_llm_apps/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.4')

### Pinecone indexes

In [4]:
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ...')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')


Index langchain-pinecone already exists!


In [5]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [15]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

### Creating indexes

In [7]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')

In [8]:
# creating...
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [9]:
# updating...
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

In [10]:
# fetching...
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'metadata': {},
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

In [11]:
# deleting...
index.delete(ids=['b', 'c'])

# to delete all
# index.delete(delete_all=True)

{}

In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [30]:
# quering vectors
vectors = [[random.random() for _ in range(1536)] for v in range(1)]

index.query(
    vector=vectors,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'e', 'score': 0.750360608, 'values': []},
             {'id': 'd', 'score': 0.742764354, 'values': []},
             {'id': 'a', 'score': 0.741675377, 'values': []}],
 'namespace': ''}